In [1]:
Kaggle = False
Colab = True

In [2]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"

    os.chdir(path)
    os.listdir(path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#  if Kaggle:
#     !rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
#     !git clone --recursive https://github.com/Microsoft/LightGBM
#     !apt-get install -y -qq libboost-all-dev
#     %%bash
#     cd LightGBM
#     rm -r build
#     mkdir build
#     cd build
#     cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
#     make -j$(nproc)
#     !cd LightGBM/python-package/;python3 setup.py install --precompile
#     !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
#     !rm -r LightGBM
# else:

# !git clone — recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install — precompile

In [4]:
# !git clone https://github.com/Microsoft/LightGBM
# %cd /content/drive/My\ Drive/LightGBM/
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [5]:
%cd /content/drive/My\ Drive/LightGBM/python-package
!sudo python setup.py install

/content/drive/My Drive/LightGBM/python-package
running install
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
copying build/lib/lightgbm/VERSION.txt -> /usr/local/lib/python3.6/dist-packages/lightgbm
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so', 'compile/lib_lightgbm.so']
running install_egg_info
Copying lightgbm.egg-info to /u

In [6]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Sat Jun 20 22:14:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [9]:
if Kaggle:
    PATH = '/kaggle/input/'
    outdir = '.'
# PATH = '/Users/helen/Desktop/Data/'
else:
    PATH = 'm5_competition/'
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/lgbm')
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [10]:
MODELNAME = 'LGBM-GPU'
VERSION = '{}'.format(MODELNAME)
outdir = os.path.join(outdir, VERSION)
if not os.path.exists(outdir):
    os.mkdir(outdir)

from datetime import datetime
dateTimeObj = datetime.now()
# timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")
# model_path = os.path.join(outdir, 'all-v2')
timestampStr = 'all-v2-l10_28-w10_28-ln256'
outdir = os.path.join(outdir, timestampStr)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [11]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "int16", 'snap_TX': 'int16', 'snap_WI': 'int16' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [12]:
pd.options.display.max_columns = 50

In [13]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [14]:

def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int16).min
                      and c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df


In [15]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv(PATH+ "m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(PATH+"m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    display(cal.head())   
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    # dtype.update({col: "category" for col in catcols})
    dt = pd.read_csv(PATH+"m5-forecasting-accuracy/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()

    # for col in catcols:
    #     dt[col] = dt[col].cat.codes.astype("int16")
    #     dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    # print("step 0")
    # display(dt.head())
    dt = dt.merge(cal, on= "d", copy = False)
    # print("step 1")
    # dt['d'] = dt['d'].astype('category').cat.codes.astype("int16")
    # print(dt['d'].min(), dt['d'].max())
    # print(np.unique(dt['d']))
    # print(len(np.unique(dt['d'])))
    # dt['d'] -= dt['d'].min()
    # print("step 1")
    # display(dt.head())
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    display(dt.head())
    return dt

In [16]:
def create_fea(dt):
    lags = [1, 2, 3, 7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [17]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [18]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df = reduce_mem_usage(df)
df.shape

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.77


Mem. usage decreased to 2718.23 MB (5.4 % reduction)
CPU times: user 27.7 s, sys: 2.36 s, total: 30.1 s
Wall time: 36 s


In [19]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       int16         
 19  snap_TX       int16         
 20  snap_WI       int16         
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(2), int16(17), object(2)
memory us

In [21]:
%%time

create_fea(df)
df.shape
df = reduce_mem_usage(df)

Mem. usage decreased to 4116.18 MB (0.0 % reduction)
CPU times: user 6min 10s, sys: 1.81 s, total: 6min 11s
Wall time: 6min 12s


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 40 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       int16         
 19  snap_TX       int16         
 20  snap_WI       int16         
 21  sell_price    float16       
 22  lag_1         float16       
 23  lag_2         float16       
 

In [23]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_1,lag_2,lag_3,lag_7,lag_28,rmean_1_7,rmean_2_7,rmean_3_7,rmean_7_7,rmean_28_7,rmean_1_28,rmean_2_28,rmean_3_28,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [24]:
df.dropna(inplace = True)
df.shape

(39041269, 40)

In [25]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [26]:
display(X_train.head())


,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_1,lag_2,lag_3,lag_7,lag_28,rmean_1_7,rmean_2_7,rmean_3_7,rmean_7_7,rmean_28_7,rmean_1_28,rmean_2_28,rmean_3_28,rmean_7_28,rmean_28_28,week,quarter,mday
869062,1,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,3.970703,0.0,0.0,0.0,0.0,0.0,0.142822,0.142822,0.142822,0.000000,0.142822,0.214233,0.214233,0.214233,0.214233,0.214233,10,1,8
869063,1,0,0,0,0,7,3,2012,0,0,0,0,1,1,1,3.970703,0.0,0.0,0.0,0.0,1.0,0.142822,0.142822,0.142822,0.000000,0.142822,0.214233,0.214233,0.214233,0.178589,0.250000,10,1,9
869069,3,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,4.339844,2.0,2.0,1.0,3.0,2.0,1.571289,1.286133,1.000000,1.857422,1.142578,1.463867,1.428711,1.357422,1.392578,1.678711,10,1,8
869070,3,0,0,0,0,7,3,2012,0,0,0,0,1,1,1,4.339844,1.0,2.0,2.0,1.0,0.0,1.286133,1.571289,1.286133,2.000000,1.000000,1.428711,1.463867,1.428711,1.392578,1.607422,10,1,9
869076,4,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,2.980469,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.857422,0.285645,0.357178,0.392822,0.750000,0.643066,10,1,8


In [27]:
display(y_train.head())

869062    0.0
869063    0.0
869069    1.0
869070    2.0
869076    0.0
Name: sales, dtype: float16

In [28]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [29]:
%%time

np.random.seed(777)

# fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
# train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
# train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
#                          categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
#                               categorical_feature=cat_feats,
#                  free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds],free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 17.1 s, sys: 838 ms, total: 17.9 s
Wall time: 17.9 s


In [30]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [31]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 4,
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 2000,
    'num_leaves': 128,
    "min_data_in_leaf": 100,
    'device': 'gpu'
}

In [32]:
%%time

m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

/content/drive/My Drive/LightGBM/python-package/lightgbm/engine.py:150: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3699
[LightGBM] [Info] Number of data points in the train set: 37041269, number of used features: 34
[LightGBM] [Info] Using GPU Device: Tesla P100-PCIE-16GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 22 dense feature groups (847.81 MB) transferred to GPU in 1.024019 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 0.303982
[20]	valid_0's rmse: 3.43327
[40]	valid_0's rmse: 2.64099
[60]	valid_0's rmse: 2.38636
[80]	valid_0's rmse: 2.30867
[100]	valid_0's rmse: 2.28612
[120]	valid_0's rmse: 2.27283
[140]	valid_0's rmse: 2.26424
[160]	valid_0's rmse: 2.25712
[180]	valid_0's rmse: 2.2497
[200]	valid_0's rmse: 2.24461
[220]	valid_0's rmse: 2.23773
[240]	valid_0's rmse: 2.2342
[260]	valid_0's rmse: 2.2289
[280]	valid_0's rmse: 2.22354
[300]

In [33]:
m_lgb.save_model(os.path.join(outdir,"model-l256-fd{}.lgb".format(FIRST_DAY)))

In [34]:
# m_lgb.load_model(os.path.join(model_path,"model-l256-fd{}.lgb".format(FIRST_DAY)))
# print("model loaded")

In [35]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

for icount, (alpha, weight) in enumerate(zip(alphas, weights)):

    te = create_dt(False)
    cols = [f"F{i}" for i in range(1,29)]

    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        create_fea(tst)
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(os.path.join(outdir, f"submission_{icount}.csv"),index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(os.path.join(outdir,"submission.csv"),index=False)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
1 1.023 0.3333333333333333


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
2 1.018 0.3333333333333333
CPU times: user 4h 20min 57s, sys: 53.2 s, total: 4h 21min 50s
Wall time: 4h 12min 23s


In [36]:
sub.head(10)

F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0.965366,0.944936,0.928302,0.954622,1.184508,1.613677,1.312262,1.109797,1.053193,1.113424,1.105534,1.264210,1.524821,1.322241,1.113164,1.048817,1.073583,1.112827,1.234643,1.499864,1.382033,1.179835,1.024817,1.008540,1.038522,1.200558,1.469733,1.474477
1,FOODS_1_001_CA_2_validation,1.178510,1.191940,1.114167,1.258095,1.357256,2.395540,1.856878,1.211609,1.250692,1.192746,1.211221,1.409056,1.949190,1.620133,1.336494,1.295392,1.373770,1.418947,1.647617,2.061759,2.191578,1.294628,1.313132,1.329508,1.415151,1.512066,1.977657,1.881962
2,FOODS_1_001_CA_3_validation,0.885656,1.045753,1.001181,1.032660,1.086794,1.522957,1.263027,1.064273,1.057645,1.020859,1.045728,1.185216,1.481956,1.355490,1.202418,1.179003,1.148688,1.179483,1.252679,1.436686,1.634026,1.217679,1.178329,1.154634,1.167565,1.274704,1.313488,1.346323
3,FOODS_1_001_CA_4_validation,0.491402,0.448242,0.447061,0.455587,0.500257,0.620300,0.552068,0.415114,0.438472,0.429613,0.443357,0.405148,0.453325,0.421733,0.397903,0.396759,0.413316,0.477874,0.490044,0.537508,0.536131,0.484803,0.434177,0.442624,0.450791,0.486445,0.511955,0.533431
4,FOODS_1_001_TX_1_validation,0.088778,0.356134,0.379361,0.399389,0.478574,0.727415,0.530711,0.451897,0.462666,0.439576,0.462673,0.481372,0.528840,0.417406,0.371357,0.355287,0.379437,0.375537,0.402624,0.358583,0.370844,0.310446,0.302611,0.299621,0.316438,0.334563,0.375479,0.404836
5,FOODS_1_001_TX_2_validation,0.439116,0.447068,0.473772,0.447249,0.438297,0.576886,0.519801,0.423067,0.456426,0.431875,0.510625,0.498244,0.555048,0.502345,0.491987,0.474255,0.518070,0.503164,0.573912,0.551200,0.595577,0.444001,0.437614,0.445026,0.470158,0.481869,0.526400,0.517413
6,FOODS_1_001_TX_3_validation,0.405023,0.422814,0.437538,0.462076,0.530927,0.650364,0.591470,0.498676,0.555065,0.513732,0.516610,0.585143,0.565676,0.498427,0.512289,0.494610,0.530042,0.577700,0.581732,0.581784,0.607026,0.509912,0.488715,0.487322,0.522450,0.544430,0.595697,0.581545
7,FOODS_1_001_WI_1_validation,0.524842,0.641323,0.658265,0.789035,0.837301,1.070839,0.692791,0.694718,0.647703,0.669843,0.875473,0.830556,0.897638,0.705055,0.664410,0.656351,0.706100,0.781553,0.895535,1.065067,0.973333,0.749826,0.748878,0.747043,0.780697,0.880999,1.220872,0.915632
8,FOODS_1_001_WI_2_validation,0.298632,0.329274,0.337947,0.347427,0.391629,0.510127,0.407716,0.422759,0.469299,0.433222,0.452531,0.470609,0.461271,0.392125,0.497987,0.386448,0.513684,0.508595,0.487597,0.528114,0.518094,0.393810,0.411513,0.399435,0.406967,0.414952,0.442806,0.429071
9,FOODS_1_001_WI_3_validation,0.268601,0.283093,0.299637,0.319882,0.361400,0.441857,0.319869,0.367246,0.413236,0.353090,0.373036,0.428447,0.417669,0.384098,0.375917,0.344470,0.378702,0.392456,0.401824,0.466723,0.439994,0.355879,0.349750,0.359513,0.382171,0.432585,0.503470,0.410743


In [37]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [38]:
sub.shape

(60980, 29)

In [39]:
print(outdir)

m5_competition/res/lgbm/LGBM-GPU/all-v2-l10_28-w10_28-ln256
